# Lecture 2 - 2

We are going to use data on NYC (Paris) subway system to find the shortest path from 2 stations. The nodes are the subway station, stored in `nodes.csv` and the arcs are the distance / time between connecting nodes, stored in `arcs.csv`.

In [1]:
library("gurobi")
library("Matrix")
library("magick")
city = "NYC"  # 'Paris'

Loading required package: slam
Warning message:
"package 'slam' was built under R version 3.5.2"Warning message:
"package 'magick' was built under R version 3.5.2"Linking to ImageMagick 6.9.9.14
Enabled features: cairo, freetype, fftw, ghostscript, lcms, pango, rsvg, webp
Disabled features: fontconfig, x11


Let's load up and examine our data

In [2]:
thePath = getwd()
arcs = as.matrix(read.csv(paste0(thePath, "/", city, "/arcs.csv"), sep = ",", header = TRUE))  # loads the arc data
nodes = as.matrix(read.csv(paste0(thePath, "/", city, "/nodes.csv"), sep = ",", header = TRUE))  # loads the nodes data
head(arcs)

from_stop_nb,to_stop_nb,dis_line,min_time_elapsed,from_stop_id,to_stop_id,route_id
185,10,1.500000e+04,0.18,A09,112,
198,23,1.500000e+04,0.18,A24,125,
448,25,1.500000e+04,0.18,R16,127,
179,25,1.500000e+04,0.18,902,127,
200,25,2.500000e+04,0.30,A27,127,
176,25,1.500000e+04,0.18,725,127,


In [3]:
options(repr.matrix.max.rows = 600, repr.matrix.max.cols = 20)  # to show the entire matrix (defaults are 60 and 20) so you can find your stops
nodes

stop_name,flow,stop_lat,stop_lon,stop_id,stop_nb,route_id
Van Cortlandt Park - 242 St,1000,40.88925,-73.89858,101,1,(1)
238 St,1000,40.88467,-73.90087,103,2,(1)
231 St,1000,40.87886,-73.90483,104,3,(1)
Marble Hill - 225 St,1000,40.87456,-73.90983,106,4,(1)
215 St,1000,40.86944,-73.91528,107,5,(1)
207 St,1000,40.86462,-73.91882,108,6,(1)
Dyckman St,1000,40.86053,-73.92554,109,7,(1)
191 St,1000,40.85522,-73.92941,110,8,(1)
181 St,1000,40.84951,-73.93360,111,9,(1)
168 St - Washington Hts,1000,40.84056,-73.94013,112,10,(1)


Now let's get things into a form that we can use! The problem that we are trying to solve is 
\begin{align}
\min_{\pi \geq 0} \sum_{(x,y) \in \mathcal{A}} \pi_{xy} c_{xy} \\
\text{s.t. } \nabla^T \pi = n 
\end{align}

Suppose that the network has `nbNodes` nodes and `nbArcs` arcs.
* $\pi$ is our object of interest,
* $c$ a vector of length `nbArcs` such that $c_a$ is the transportation cost at arc $a$,
* $\nabla$ is an matrix of size `nbArcs` $\times$ `nbNodes`. If the first arc has $i$ as the origin node and $j$ as the the destination node then $\nabla_{1i} = -1$ and $\nabla_{1j} = 1$,
* $n$, a vector of length `nbNodes` such that $n_x$ is the net demand at node $x$, i.e. $n_i = -1$, $n_j = 1$;

In [4]:
originNode = 383  # Union Sq. on the L train
destinationNode = 394  # Myrtle Wyckoff on the L train

In [5]:
nbNodes = max(as.numeric(arcs[, 1]))
nbArcs = dim(arcs)[1]
namesNodes = nodes[, 1]
c = arcs[, 3]
n = rep(0, nbNodes)  # construct vector of exiting flow, net demand is zero
n[c(originNode, destinationNode)] = c(-1, 1)  # except for our origin and destination

In [6]:
?sparseMatrix

In [7]:
Nabla = sparseMatrix(i = 1:nbArcs, j = as.numeric(arcs[, 1]), dims = c(nbArcs, nbNodes), 
    x = -1) + sparseMatrix(i = 1:nbArcs, j = as.numeric(arcs[, 2]), dims = c(nbArcs, 
    nbNodes), x = 1)

Now let's solve using Gurobi:
* `A` = $\nabla^T$
* `obj` = $c$
* `sense` = '$=$'
* `rhs` = $n$
* `modelsense` = '$\min$'.

In [8]:
result = gurobi(list(A = t(Nabla), obj = as.numeric(c), sense = "=", rhs = n, modelsense = "min", 
    start = matrix(0, nbArcs, 1)), params = NULL)
pi = result$x
distance = result$objval

Optimize a model with 501 rows, 1290 columns and 2580 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 3e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 358 rows and 716 columns
Presolve time: 0.00s
Presolved: 143 rows, 574 columns, 1148 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.000000e+00   0.000000e+00      0s
       4    8.2100056e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.00 seconds
Optimal objective  8.210005600e+03


Let's deduce the minimal distance path

In [9]:
# Some plotting stuff
themargin = -c(1, 1, 0.5, 0.2)
require("igraph")
geoCoordinates = nodes[, 3:4]
class(geoCoordinates) = "numeric"
# mapCoordinates = nodes[,5:6] class(mapCoordinates)='numeric'
nbNodes = max(arcs[, 1])
nbArcs = dim(arcs)[1]

plotCurrentNetwork = function(network, curNode) {
    sizeNodes = rep(1, nbNodes)
    sizeNodes[originNode] = 4
    sizeNodes[destinationNode] = 4
    sizeNodes[curNode] = 4
    labelNodes = rep(NA, nbNodes)
    labelNodes[originNode] = namesNodes[originNode]
    labelNodes[destinationNode] = namesNodes[destinationNode]
    labelNodes[curNode] = namesNodes[curNode]
    plot.igraph(network, vertex.label = labelNodes, vertex.label.cex = 1, vertex.size = sizeNodes, 
        edge.arrow.size = 0, layout = geoCoordinates, margin = themargin)
}

thegraph = graph_from_edgelist(arcs[, 1:2])

labelColors = rep("SkyBlue2", nbNodes)
labelColors[originNode] = "firebrick2"
labelColors[destinationNode] = "forestgreen"

sizeNodes = rep(1, nbNodes)
sizeNodes[originNode] = 4
sizeNodes[destinationNode] = 4

nbNodesSoFar = 1
curpoint = originNode

cont = TRUE
i = originNode
writeLines(paste0(namesNodes[i], " (#", i, ")"))
eqpath = which(pi > 0)
rank = 0

frames = image_graph(width = 600, height = 600, res = 150)

cont = TRUE
i = originNode
writeLines(paste0(namesNodes[i], " (#", i, ")"))
eqpath = which(pi > 0)
rank = 0
while (cont) {
    rank = rank + 1
    leavingi = which(Nabla[, i] == -1)
    a = intersect(eqpath, leavingi)[1]
    j = which(Nabla[a, ] == 1)[1]
    plotCurrentNetwork(thegraph, j)
    writeLines(paste0(rank, ": ", namesNodes[j], " (#", j, ")"))
    i = j
    if (j == destinationNode) {
        cont <- FALSE
    }
}
# done with plotting
dev.off()

Loading required package: igraph
Warning message:
"package 'igraph' was built under R version 3.5.2"
Attaching package: 'igraph'

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union



Union Sq - 14 St (#383)
Union Sq - 14 St (#383)
1: 3 Av (#384)
2: 1 Av (#385)
3: Bedford Av (#386)
4: Lorimer St (#387)
5: Graham Av (#388)
6: Grand St (#389)
7: Montrose Av (#390)
8: Morgan Av (#391)
9: Jefferson St (#392)
10: DeKalb Av (#393)
11: Myrtle - Wyckoff Avs (#394)


png 
  2

In [10]:
# animate
image_animate(frames, 1)

   format width height colorspace matte filesize density
1     gif   600    600       sRGB  TRUE        0   72x72
2     gif   600    600       sRGB  TRUE        0   72x72
3     gif   600    600       sRGB  TRUE        0   72x72
4     gif   600    600       sRGB  TRUE        0   72x72
5     gif   600    600       sRGB  TRUE        0   72x72
6     gif   600    600       sRGB  TRUE        0   72x72
7     gif   600    600       sRGB  TRUE        0   72x72
8     gif   600    600       sRGB  TRUE        0   72x72
9     gif   600    600       sRGB  TRUE        0   72x72
10    gif   600    600       sRGB  TRUE        0   72x72
11    gif   600    600       sRGB  TRUE        0   72x72